# 🧪 Đánh giá các phương pháp ML/DL cho ABSA (UIT-ViSFD)

Notebook này đánh giá các phương pháp **Machine Learning truyền thống** và **Deep Learning** cho tác vụ **Aspect-Based Sentiment Analysis**.

## Các phương pháp được đánh giá:

### Machine Learning truyền thống:
- **Naive Bayes** (MultinomialNB)
- **SVM** (Support Vector Machine với kernel linear/rbf)
- **Random Forest**

### Deep Learning:
- **CNN** (Convolutional Neural Network)
- **LSTM** (Long Short-Term Memory)

## Metrics đánh giá:
- Precision, Recall, F1-Score (macro, micro, weighted)
- Confusion Matrix
- Classification Report chi tiết theo từng class

> **Lưu ý**: Notebook này tập trung vào **Sentiment Classification** trên các khía cạnh đã được trích xuất.

In [ ]:
# ========================
# 1) Thiết lập đường dẫn & tham số
# ========================
GROUND_TRUTH_CSV = "/Users/hatrungkien/my-sentiment/data/raw/UIT-ViSFD/Test.csv"
PREDICTIONS_CSV = "/Users/hatrungkien/my-sentiment/outputs/absa/openai/uit/uit_absa_2225.csv"

# Bỏ qua các khía cạnh tổng quát
IGNORE_ASPECTS = {"GENERAL", "OTHERS"}

# Random seed cho reproducibility
RANDOM_STATE = 42

# Tham số cho Deep Learning
MAX_SEQUENCE_LENGTH = 128
EMBEDDING_DIM = 100
BATCH_SIZE = 32
EPOCHS = 10

# Test size cho train/test split
TEST_SIZE = 0.2

In [ ]:
# ========================
# 2) Import thư viện
# ========================
import warnings
warnings.filterwarnings('ignore')

import re
import ast
import json
from collections import Counter, defaultdict
from typing import Dict, List, Set, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn imports
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    precision_recall_fscore_support,
    f1_score
)

# Traditional ML models
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Deep Learning imports
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import (
        Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout,
        LSTM, Bidirectional, SpatialDropout1D
    )
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.callbacks import EarlyStopping
    DL_AVAILABLE = True
except ImportError:
    print("⚠️ TensorFlow không được cài đặt. Các mô hình Deep Learning sẽ bị bỏ qua.")
    DL_AVAILABLE = False

print("✅ Import thư viện thành công!")

In [ ]:
# ========================
# 3) Các hàm tiện ích
# ========================

def normalize_sentiment(s):
    """Đưa sentiment về một trong {Positive, Negative, Neutral}."""
    if s is None:
        return None
    s = str(s).strip().lower()
    table = {
        "pos": "Positive", "positive": "Positive", "+": "Positive",
        "neg": "Negative", "negative": "Negative", "-": "Negative",
        "neu": "Neutral",  "neutral":  "Neutral",  "0": "Neutral",
    }
    return table.get(s, s.capitalize())

# Bảng alias cho khía cạnh
ASPECT_ALIASES = {
    "screen": "SCREEN", "display": "SCREEN", "màn hình": "SCREEN",
    "battery": "BATTERY", "pin": "BATTERY",
    "camera": "CAMERA", "cam": "CAMERA",
    "storage": "STORAGE", "memory": "STORAGE", "rom": "STORAGE", "ram": "STORAGE",
    "design": "DESIGN", "thiết kế": "DESIGN",
    "performance": "PERFORMANCE", "speed": "PERFORMANCE", "hiệu năng": "PERFORMANCE",
    "ser&acc": "SER&ACC", "service": "SER&ACC", "support": "SER&ACC",
    "features": "FEATURES", "setup": "FEATURES", "tính năng": "FEATURES",
    "sound": "SOUND", "audio": "SOUND", "loa": "SOUND",
    "price": "PRICE", "giá": "PRICE",
    "general": "GENERAL", "others": "OTHERS", "other": "OTHERS"
}

def canonicalize_aspect(a):
    """Chuẩn hóa tên khía cạnh."""
    if a is None:
        return None
    a0 = str(a).strip().strip('"\'').lower()
    a0 = a0.replace("&amp;", "&")
    a0 = re.sub(r"\s+", " ", a0)
    return ASPECT_ALIASES.get(a0, a0.upper())

def parse_gt_labels(label_str: str) -> Dict[str, str]:
    """Parse ground truth labels từ định dạng UIT-ViSFD."""
    result = {}
    if pd.isna(label_str) or not str(label_str).strip():
        return result
    parts = str(label_str).split(";")
    for item in parts:
        item = item.strip().strip("{} ")
        if not item:
            continue
        if "#" in item:
            asp, sent = item.split("#", 1)
            asp = canonicalize_aspect(asp)
            sent = normalize_sentiment(sent)
        else:
            asp = canonicalize_aspect(item)
            sent = "Neutral"
        if asp:
            result[asp] = sent
    return result

def clean_text(text):
    """Làm sạch văn bản tiếng Việt."""
    if pd.isna(text):
        return ""
    text = str(text).lower().strip()
    # Giữ lại ký tự tiếng Việt và số
    text = re.sub(r'[^\w\s.,!?àáảãạăằắẳẵặâầấẩẫậèéẻẽẹêềếểễệìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵđ]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

print("✅ Định nghĩa các hàm tiện ích thành công!")

In [ ]:
# ========================
# 4) Đọc và xử lý dữ liệu
# ========================

# Đọc ground truth
gt = pd.read_csv(GROUND_TRUTH_CSV)
print(f"Ground truth shape: {gt.shape}")
print(f"Columns: {gt.columns.tolist()}")

# Tìm cột ID và cột text
id_col = 'index' if 'index' in gt.columns else gt.columns[0]
text_col = 'comment' if 'comment' in gt.columns else 'sentence'
label_col = 'label' if 'label' in gt.columns else gt.columns[-1]

gt['review_id'] = gt[id_col]
gt['text'] = gt[text_col]

print(f"\nSố lượng reviews: {len(gt)}")
display(gt.head(3))

In [ ]:
# ========================
# 5) Tạo dataset cho training
# ========================

# Tạo dữ liệu ở cấp (text, aspect, sentiment)
data_records = []

for idx, row in gt.iterrows():
    text = clean_text(row['text'])
    if not text:
        continue
    
    labels = parse_gt_labels(row[label_col])
    
    for aspect, sentiment in labels.items():
        if aspect in IGNORE_ASPECTS:
            continue
        if sentiment not in ['Positive', 'Negative', 'Neutral']:
            continue
        
        # Tạo text kết hợp aspect
        combined_text = f"{text} [ASPECT: {aspect}]"
        
        data_records.append({
            'review_id': row['review_id'],
            'text': text,
            'aspect': aspect,
            'combined_text': combined_text,
            'sentiment': sentiment
        })

df_data = pd.DataFrame(data_records)
print(f"Tổng số samples: {len(df_data)}")
print(f"\nPhân bố sentiment:")
print(df_data['sentiment'].value_counts())
print(f"\nPhân bố aspect:")
print(df_data['aspect'].value_counts())

In [ ]:
# ========================
# 6) Chia train/test và chuẩn bị features
# ========================

# Encode labels
label_encoder = LabelEncoder()
df_data['sentiment_encoded'] = label_encoder.fit_transform(df_data['sentiment'])

print(f"Label mapping: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")

# Chia train/test
X_text = df_data['combined_text'].values
y = df_data['sentiment_encoded'].values

X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE,
    stratify=y
)

print(f"\nTrain size: {len(X_train_text)}")
print(f"Test size: {len(X_test_text)}")

# TF-IDF Vectorizer cho ML truyền thống
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95
)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)
X_test_tfidf = tfidf_vectorizer.transform(X_test_text)

print(f"\nTF-IDF features shape: {X_train_tfidf.shape}")

In [ ]:
# ========================
# 7) Hàm đánh giá và hiển thị kết quả
# ========================

def evaluate_model(y_true, y_pred, model_name, label_names=None):
    """Đánh giá và hiển thị kết quả của model."""
    print(f"\n{'='*60}")
    print(f"📊 Kết quả: {model_name}")
    print(f"{'='*60}")
    
    # Accuracy
    acc = accuracy_score(y_true, y_pred)
    print(f"\n🎯 Accuracy: {acc:.4f}")
    
    # Precision, Recall, F1
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro'
    )
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        y_true, y_pred, average='micro'
    )
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        y_true, y_pred, average='weighted'
    )
    
    print(f"\n📈 Metrics (Macro):")
    print(f"   Precision: {precision_macro:.4f}")
    print(f"   Recall:    {recall_macro:.4f}")
    print(f"   F1-Score:  {f1_macro:.4f}")
    
    print(f"\n📈 Metrics (Micro):")
    print(f"   Precision: {precision_micro:.4f}")
    print(f"   Recall:    {recall_micro:.4f}")
    print(f"   F1-Score:  {f1_micro:.4f}")
    
    print(f"\n📈 Metrics (Weighted):")
    print(f"   Precision: {precision_weighted:.4f}")
    print(f"   Recall:    {recall_weighted:.4f}")
    print(f"   F1-Score:  {f1_weighted:.4f}")
    
    # Classification Report
    print(f"\n📋 Classification Report:")
    print(classification_report(y_true, y_pred, target_names=label_names))
    
    # Return metrics dict
    return {
        'model': model_name,
        'accuracy': acc,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'precision_micro': precision_micro,
        'recall_micro': recall_micro,
        'f1_micro': f1_micro,
        'precision_weighted': precision_weighted,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted
    }

def plot_confusion_matrix(y_true, y_pred, label_names, model_name):
    """Vẽ confusion matrix."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=label_names, yticklabels=label_names)
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.tight_layout()
    plt.show()

print("✅ Định nghĩa các hàm đánh giá thành công!")

---
# 🤖 Phần 1: Machine Learning Truyền Thống
---

In [ ]:
# ========================
# 8) Naive Bayes
# ========================
print("\n" + "="*60)
print("🔹 Training Naive Bayes (MultinomialNB)...")
print("="*60)

nb_model = MultinomialNB(alpha=1.0)
nb_model.fit(X_train_tfidf, y_train)

# Predict
y_pred_nb = nb_model.predict(X_test_tfidf)

# Evaluate
nb_metrics = evaluate_model(
    y_test, y_pred_nb, 
    "Naive Bayes (MultinomialNB)",
    label_encoder.classes_
)

# Confusion Matrix
plot_confusion_matrix(y_test, y_pred_nb, label_encoder.classes_, "Naive Bayes")

In [ ]:
# ========================
# 9) Support Vector Machine (SVM)
# ========================
print("\n" + "="*60)
print("🔹 Training SVM (Linear Kernel)...")
print("="*60)

svm_model = SVC(
    kernel='linear',
    C=1.0,
    random_state=RANDOM_STATE,
    class_weight='balanced'
)
svm_model.fit(X_train_tfidf, y_train)

# Predict
y_pred_svm = svm_model.predict(X_test_tfidf)

# Evaluate
svm_metrics = evaluate_model(
    y_test, y_pred_svm,
    "SVM (Linear Kernel)",
    label_encoder.classes_
)

# Confusion Matrix
plot_confusion_matrix(y_test, y_pred_svm, label_encoder.classes_, "SVM")

In [ ]:
# ========================
# 10) Random Forest
# ========================
print("\n" + "="*60)
print("🔹 Training Random Forest...")
print("="*60)

rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=RANDOM_STATE,
    class_weight='balanced',
    n_jobs=-1
)
rf_model.fit(X_train_tfidf, y_train)

# Predict
y_pred_rf = rf_model.predict(X_test_tfidf)

# Evaluate
rf_metrics = evaluate_model(
    y_test, y_pred_rf,
    "Random Forest",
    label_encoder.classes_
)

# Confusion Matrix
plot_confusion_matrix(y_test, y_pred_rf, label_encoder.classes_, "Random Forest")

---
# 🧠 Phần 2: Deep Learning
---

In [ ]:
# ========================
# 11) Chuẩn bị dữ liệu cho Deep Learning
# ========================

if DL_AVAILABLE:
    print("\n" + "="*60)
    print("🔧 Chuẩn bị dữ liệu cho Deep Learning...")
    print("="*60)
    
    # Tokenizer
    tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
    tokenizer.fit_on_texts(X_train_text)
    
    # Convert to sequences
    X_train_seq = tokenizer.texts_to_sequences(X_train_text)
    X_test_seq = tokenizer.texts_to_sequences(X_test_text)
    
    # Pad sequences
    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
    X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
    
    # One-hot encode labels
    num_classes = len(label_encoder.classes_)
    y_train_cat = to_categorical(y_train, num_classes)
    y_test_cat = to_categorical(y_test, num_classes)
    
    vocab_size = min(10000, len(tokenizer.word_index) + 1)
    
    print(f"Vocabulary size: {vocab_size}")
    print(f"Max sequence length: {MAX_SEQUENCE_LENGTH}")
    print(f"Number of classes: {num_classes}")
    print(f"X_train_pad shape: {X_train_pad.shape}")
    print(f"X_test_pad shape: {X_test_pad.shape}")
else:
    print("⚠️ TensorFlow không khả dụng. Bỏ qua Deep Learning.")

In [ ]:
# ========================
# 12) CNN Model
# ========================

if DL_AVAILABLE:
    print("\n" + "="*60)
    print("🔹 Training CNN (Convolutional Neural Network)...")
    print("="*60)
    
    # Build CNN model
    cnn_model = Sequential([
        Embedding(vocab_size, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH),
        SpatialDropout1D(0.2),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    cnn_model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print(cnn_model.summary())
    
    # Early stopping
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    
    # Train
    history_cnn = cnn_model.fit(
        X_train_pad, y_train_cat,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.1,
        callbacks=[early_stop],
        verbose=1
    )
    
    # Predict
    y_pred_cnn_proba = cnn_model.predict(X_test_pad)
    y_pred_cnn = np.argmax(y_pred_cnn_proba, axis=1)
    
    # Evaluate
    cnn_metrics = evaluate_model(
        y_test, y_pred_cnn,
        "CNN",
        label_encoder.classes_
    )
    
    # Confusion Matrix
    plot_confusion_matrix(y_test, y_pred_cnn, label_encoder.classes_, "CNN")
    
    # Plot training history
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    axes[0].plot(history_cnn.history['accuracy'], label='Train Accuracy')
    axes[0].plot(history_cnn.history['val_accuracy'], label='Val Accuracy')
    axes[0].set_title('CNN - Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    
    axes[1].plot(history_cnn.history['loss'], label='Train Loss')
    axes[1].plot(history_cnn.history['val_loss'], label='Val Loss')
    axes[1].set_title('CNN - Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
else:
    cnn_metrics = None
    print("⚠️ Bỏ qua CNN do TensorFlow không khả dụng.")

In [ ]:
# ========================
# 13) LSTM Model
# ========================

if DL_AVAILABLE:
    print("\n" + "="*60)
    print("🔹 Training LSTM (Long Short-Term Memory)...")
    print("="*60)
    
    # Build LSTM model
    lstm_model = Sequential([
        Embedding(vocab_size, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH),
        SpatialDropout1D(0.2),
        Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    lstm_model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print(lstm_model.summary())
    
    # Early stopping
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    
    # Train
    history_lstm = lstm_model.fit(
        X_train_pad, y_train_cat,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.1,
        callbacks=[early_stop],
        verbose=1
    )
    
    # Predict
    y_pred_lstm_proba = lstm_model.predict(X_test_pad)
    y_pred_lstm = np.argmax(y_pred_lstm_proba, axis=1)
    
    # Evaluate
    lstm_metrics = evaluate_model(
        y_test, y_pred_lstm,
        "LSTM (Bidirectional)",
        label_encoder.classes_
    )
    
    # Confusion Matrix
    plot_confusion_matrix(y_test, y_pred_lstm, label_encoder.classes_, "LSTM")
    
    # Plot training history
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    axes[0].plot(history_lstm.history['accuracy'], label='Train Accuracy')
    axes[0].plot(history_lstm.history['val_accuracy'], label='Val Accuracy')
    axes[0].set_title('LSTM - Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    
    axes[1].plot(history_lstm.history['loss'], label='Train Loss')
    axes[1].plot(history_lstm.history['val_loss'], label='Val Loss')
    axes[1].set_title('LSTM - Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
else:
    lstm_metrics = None
    print("⚠️ Bỏ qua LSTM do TensorFlow không khả dụng.")

---
# 📊 Phần 3: So sánh tổng hợp các phương pháp
---

In [ ]:
# ========================
# 14) Tổng hợp và so sánh kết quả
# ========================

# Collect all metrics
all_metrics = [nb_metrics, svm_metrics, rf_metrics]
if DL_AVAILABLE and cnn_metrics:
    all_metrics.append(cnn_metrics)
if DL_AVAILABLE and lstm_metrics:
    all_metrics.append(lstm_metrics)

# Create comparison dataframe
comparison_df = pd.DataFrame(all_metrics)
comparison_df = comparison_df.set_index('model')

print("\n" + "="*80)
print("📊 BẢNG SO SÁNH TỔNG HỢP CÁC PHƯƠNG PHÁP")
print("="*80)

# Display formatted
display_cols = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro', 
                'precision_weighted', 'recall_weighted', 'f1_weighted']
display(comparison_df[display_cols].round(4))

In [ ]:
# ========================
# 15) Visualize comparison
# ========================

# Plot F1 scores comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# F1 Macro comparison
models = comparison_df.index.tolist()
f1_macro = comparison_df['f1_macro'].values
f1_weighted = comparison_df['f1_weighted'].values

x = np.arange(len(models))
width = 0.35

bars1 = axes[0].bar(x - width/2, f1_macro, width, label='F1 Macro')
bars2 = axes[0].bar(x + width/2, f1_weighted, width, label='F1 Weighted')

axes[0].set_xlabel('Model')
axes[0].set_ylabel('F1 Score')
axes[0].set_title('So sánh F1 Score giữa các phương pháp')
axes[0].set_xticks(x)
axes[0].set_xticklabels(models, rotation=45, ha='right')
axes[0].legend()
axes[0].set_ylim(0, 1)

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    axes[0].annotate(f'{height:.3f}',
                    xy=(bar.get_x() + bar.get_width()/2, height),
                    xytext=(0, 3), textcoords="offset points",
                    ha='center', va='bottom', fontsize=8)

for bar in bars2:
    height = bar.get_height()
    axes[0].annotate(f'{height:.3f}',
                    xy=(bar.get_x() + bar.get_width()/2, height),
                    xytext=(0, 3), textcoords="offset points",
                    ha='center', va='bottom', fontsize=8)

# Accuracy comparison
accuracy = comparison_df['accuracy'].values
bars3 = axes[1].bar(models, accuracy, color='steelblue')
axes[1].set_xlabel('Model')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('So sánh Accuracy giữa các phương pháp')
axes[1].set_xticklabels(models, rotation=45, ha='right')
axes[1].set_ylim(0, 1)

for bar in bars3:
    height = bar.get_height()
    axes[1].annotate(f'{height:.3f}',
                    xy=(bar.get_x() + bar.get_width()/2, height),
                    xytext=(0, 3), textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# ========================
# 16) Radar Chart so sánh
# ========================

from math import pi

# Chọn metrics để so sánh
categories = ['Accuracy', 'Precision\n(Macro)', 'Recall\n(Macro)', 'F1\n(Macro)']
N = len(categories)

# Tạo angles
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

# Plot
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

colors = plt.cm.Set2(np.linspace(0, 1, len(comparison_df)))

for idx, (model_name, row) in enumerate(comparison_df.iterrows()):
    values = [row['accuracy'], row['precision_macro'], 
              row['recall_macro'], row['f1_macro']]
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, label=model_name, color=colors[idx])
    ax.fill(angles, values, alpha=0.1, color=colors[idx])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)
ax.set_ylim(0, 1)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
ax.set_title('So sánh hiệu năng các phương pháp\n(Radar Chart)', size=14, y=1.08)

plt.tight_layout()
plt.show()

In [ ]:
# ========================
# 17) Lưu kết quả
# ========================
from pathlib import Path

out_dir = Path("absa_ml_dl_eval_reports")
out_dir.mkdir(parents=True, exist_ok=True)

# Save comparison table
comparison_df.to_csv(out_dir / "model_comparison.csv")

# Save detailed results as JSON
results_summary = {
    'config': {
        'ground_truth_csv': GROUND_TRUTH_CSV,
        'predictions_csv': PREDICTIONS_CSV,
        'ignore_aspects': list(IGNORE_ASPECTS),
        'test_size': TEST_SIZE,
        'random_state': RANDOM_STATE,
        'max_sequence_length': MAX_SEQUENCE_LENGTH,
        'embedding_dim': EMBEDDING_DIM,
        'batch_size': BATCH_SIZE,
        'epochs': EPOCHS
    },
    'results': all_metrics,
    'best_model': comparison_df['f1_macro'].idxmax(),
    'best_f1_macro': float(comparison_df['f1_macro'].max())
}

with open(out_dir / "results_summary.json", "w", encoding="utf-8") as f:
    json.dump(results_summary, f, ensure_ascii=False, indent=2)

print(f"\n✅ Đã lưu kết quả vào thư mục: {out_dir}")
print(f"   - model_comparison.csv")
print(f"   - results_summary.json")

In [ ]:
# ========================
# 18) Kết luận
# ========================

print("\n" + "="*80)
print("📋 KẾT LUẬN")
print("="*80)

best_model = comparison_df['f1_macro'].idxmax()
best_f1 = comparison_df.loc[best_model, 'f1_macro']
best_acc = comparison_df.loc[best_model, 'accuracy']

print(f"\n🏆 Mô hình tốt nhất (theo F1 Macro): {best_model}")
print(f"   - F1 Macro:  {best_f1:.4f}")
print(f"   - Accuracy:  {best_acc:.4f}")

print("\n📊 Xếp hạng các mô hình (theo F1 Macro):")
ranking = comparison_df['f1_macro'].sort_values(ascending=False)
for i, (model, score) in enumerate(ranking.items(), 1):
    print(f"   {i}. {model}: {score:.4f}")

print("\n" + "="*80)
print("📝 GHI CHÚ:")
print("- Kết quả có thể thay đổi tùy thuộc vào tham số và random seed.")
print("- Deep Learning models cần nhiều dữ liệu hơn để đạt hiệu quả tốt.")
print("- Có thể cải thiện bằng: hyperparameter tuning, pre-trained embeddings, ensemble.")
print("="*80)

---
## 📚 Phụ lục: Gợi ý cải thiện

### 1. Hyperparameter Tuning
- Sử dụng GridSearchCV hoặc RandomizedSearchCV cho ML models
- Keras Tuner cho Deep Learning models

### 2. Pre-trained Embeddings
- Word2Vec tiếng Việt
- FastText tiếng Việt
- PhoBERT (transformer-based)

### 3. Ensemble Methods
- Voting Classifier
- Stacking
- Bagging

### 4. Advanced Deep Learning
- Attention mechanisms
- Transformer-based models (BERT, PhoBERT)
- Multi-task learning

### 5. Data Augmentation
- Back-translation
- Synonym replacement
- Random insertion/deletion